In [ ]:
#!pip install renumics-spotlight datasets


Visualize with features extracted from foundational models

In [2]:
import pandas as pd
from datasets import Dataset
from renumics import spotlight

dataset_name = "fashion"

if dataset_name == 'wikiart':
    df = pd.read_pickle('DATA/Dataset/wikiart_full_combined_no_artist_filtered.pkl')
elif dataset_name == 'fashion':
    df = pd.read_pickle('DATA/Dataset/iDesigner/idesigner_influences_cropped_features.pkl')
all_artist_names = set(df['artist_name'])
df['influenced_by'] = df['influenced_by'].apply(lambda artists_list: [artist for artist in artists_list if artist in all_artist_names])

df.image_features = df.image_features.apply(lambda x: x.numpy())
if dataset_name == 'wikiart':
    df.drop(columns = ['title'],inplace=True)
    df.text_features = df.text_features.apply(lambda x: x.numpy())
    df.image_text_features = df.image_text_features.apply(lambda x: x.numpy())

dataset = Dataset.from_pandas(df)

spotlight.show(dataset, dtype={'image_features':spotlight.Embedding})


Visualize with finetuned 

In [ ]:
from Triplet_Network import TripletResNet_features
import torch
feature = 'image_features'
model_path = f'trained_models/{dataset_name}/TripletResNet_{feature}_posrandom_negrandom_10_margin2/model.pth'
model = TripletResNet_features(df.loc[0,feature].shape[0])
model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))
model.eval()
df[f'trained_{feature}'] = df[feature].apply(lambda x: model.forward_once(x).detach())
df.trained_image_text_features = df.trained_image_text_features.apply(lambda x: x.numpy())

dataset = Dataset.from_pandas(df)

spotlight.show(dataset, dtype={'trained_image_features':spotlight.Embedding})